In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
os.getcwd()

'/Users/shreyas/Desktop/Shreyas/toy_inverse_problem'

https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm


In [3]:
def read_state(file_state = 'state.txt'):
    
    with open(file_state, 'r') as reader:
        content = reader.readlines()
        M0 = float(content[0])
        M1 = float(content[1])
        
    return M0, M1

def read_gradient(file_grad = 'gradient.txt'):
    
    with open(file_grad, 'r') as reader:
        content = reader.readlines()
        M0b = float(content[0])
        M1b = float(content[1])
        
    return M0b, M1b

def write_state(M0, M1, file_state = 'state.txt'):
    
    with open(file_state, 'w') as writer:
        writer.write(f"{M0}\n")
        writer.write(f"{M1}")
        
    return None

def eval_gradient(M0, M1, file_gradient = 'gradient.txt', file_state = 'state.txt'):
    write_state(M0, M1)
    os.system("make -f Makefile clean; make -f Makefile; ./adjoint")
    
    with open(file_gradient, 'r') as reader:
        content = reader.readlines()
        M0b = float(content[0])
        M1b = float(content[1])
    
        
    return M0b, M1b

def eval_loss(M0, M1, file_loss = 'loss_inexact_line_search.txt', file_state = 'state.txt'):
    write_state(M0, M1)
    os.system("make -f Makefile_forward clean; make -f Makefile_forward; ./forward")
    
    with open(file_loss, 'r') as reader:
        content = reader.readlines()
        J = float(content[0])
        
    return J

In [4]:
def BFGS_update_matrix(B, inv_B, y, s):
    
    B_times_s = np.matmul(B, s)
    y_dot_s = np.inner(y, s)
    
    ##### UPDATE B #####
    B_new = B + np.outer(y,y)/y_dot_s - np.outer(B_times_s, B_times_s)/np.inner(s, B_times_s)
    
    ##### UPDATE INV_B #####
    I = np.eye(B.shape[0], dtype = float)
    
    left = I - np.outer(s, y)/ y_dot_s
    right = I - np.outer(y, s)/ y_dot_s
    inv_B_new = np.matmul(left, np.matmul(inv_B, right)) + np.outer(s, s) / y_dot_s
    
    eigs, _ = np.linalg.eig(B_new)
    
    for eig in eigs:
        if(eig <= 0):
            print("Hessian not Positive definite")
            break
        else:
            pass
        
    return B_new, inv_B_new

def search_alpha(state, gradient, rho, alpha = 1, c1 = 1e-4, c2 = 0.9):
    while True:
        
        f_old = eval_loss(state[0], state[1])
        f_new = eval_loss(state[0] + alpha*rho[0], state[1] + alpha*rho[1])
        
        gradient_new = np.array(eval_gradient(state[0] + alpha*rho[0], state[1] + alpha*rho[1]))
        #print((f_new - f_old) - c1*alpha*np.inner(rho, gradient), np.inner(rho, gradient_new) -  c2*np.inner(rho, gradient))
        if (f_new - f_old) <= c1*alpha*np.inner(rho, gradient) and np.inner(rho, gradient_new) >=  c2*np.inner(rho, gradient): 
            break
        else:
            alpha = alpha/2
    return alpha



https://scicomp.stackexchange.com/questions/11323/effect-of-initial-guess-b-approximate-hessian-on-bfgs-algorithm

In [ ]:
##### INITIAL GUESS #####
M0 = 0.02
M1 = 0.01
write_state(M0, M1)

MAX_ITERS = 50



state = np.array([M0, M1])
gradient = np.array(eval_gradient(M0, M1))

B = np.eye(2, dtype = float)
B_inv = np.eye(2, dtype = float)

for iteration in range(MAX_ITERS):
    
    state_old = np.copy(state)
    gradient_old = np.copy(gradient)
    
    rho = -np.matmul(B_inv, gradient_old)
    alpha = search_alpha(state_old, gradient_old, rho)
    
    s = alpha*rho
    state = state_old + s
    gradient = eval_gradient(state[0], state[1])
    y = gradient - gradient_old
        
    if iteration == 0:
        
        pass
        
    elif iteration == 1 or np.inner(y, s) <= 0:
        
        if (iteration == 1): print(y, s)
        #print(f"B reset iter {iteration}")
        B =  np.inner(y, y) / np.abs(np.inner(y, s)) * np.eye(2, dtype = float)
        B_inv =  np.abs(np.inner(y, s)) / np.inner(y, y) * np.eye(2, dtype = float)
        
    else:
            
        B, B_inv = BFGS_update_matrix(B, B_inv, y, s)
    
    print(f"iter = {iteration+1}; M0 = {state[0]}; M1 = {state[1]}; Loss = {eval_loss(state[0], state[1])}")
    

0.5
0.25
0.125
0.0625
0.03125
0.015625
0.0078125
0.00390625
0.001953125
0.0009765625
iter = 1; M0 = 0.02033114380278951; M1 = 0.0009822981888490488; Loss = 0.4963958949023225
0.5
0.25
0.125
0.0625
0.03125
0.015625
0.0078125
0.00390625
0.001953125
0.0009765625
0.00048828125
0.000244140625
0.0001220703125
6.103515625e-05
3.0517578125e-05
1.52587890625e-05
7.62939453125e-06
3.814697265625e-06
1.9073486328125e-06
9.5367431640625e-07
4.76837158203125e-07
2.384185791015625e-07
1.1920928955078125e-07
5.960464477539063e-08
[-132.25775344 2702.16337127] [-8.7075343e-06  1.6230108e-04]
iter = 2; M0 = 0.020322436268492387; M1 = 0.001144599268958216; Loss = 0.3631812199547151
iter = 3; M0 = 0.020321605375169496; M1 = 0.001145848688578427; Loss = 0.36315898320561846
iter = 4; M0 = 0.02032041543572201; M1 = 0.0011460587882240014; Loss = 0.3631455623529033
iter = 5; M0 = 0.02030040363370411; M1 = 0.0011472459471442793; Loss = 0.36296942065967674
iter = 6; M0 = 0.020258439131678112; M1 = 0.00114717541

https://www.stat.cmu.edu/~ryantibs/convexopt-F16/lectures/quasi-newton.pdf           
https://www.stat.cmu.edu/~ryantibs/convexopt-F18/lectures/quasi-newton.pdf

In [5]:
def L_BFGS_update(k, state_k, y_list, s_list, m):
    
    q = - np.array(eval_gradient(state_k[0], state_k[1]))
    
    alpha = np.zeros(k)
    
    for i in range(k-1, np.maximum(k-m, 0) - 1, -1):
        
        alpha[i] = np.inner(s_list[i-np.maximum(k-m, 0)], q)/np.inner(y_list[i-np.maximum(k-m, 0)], s_list[i-np.maximum(k-m, 0)])
        q = q - alpha[i]*y_list[i-np.maximum(k-m, 0)]
        
    p = np.inner(np.array(y_list[-1]), np.array(s_list[-1]))/np.inner(np.array(y_list[-1]), np.array(y_list[-1]))*q
    
    for i in range(np.maximum(k-m, 0), k):
        beta = np.inner(y_list[i - np.maximum(k-m, 0)], p)/np.inner(y_list[i - np.maximum(k-m, 0)], s_list[i - np.maximum(k-m, 0)])
        print(i, k-m, np.maximum(k-m, 0))
        p = p + (alpha[i] - beta)*s_list[i - np.maximum(k-m, 0)]
        
    return np.array(p)

def manage_y_s_lists(m, y_list, s_list, y, s):
    if len(y_list) < m and len(s_list) < m and len(y_list) == len(s_list):
        y_list.append(np.array(y))
        s_list.append(np.array(s))
    elif len(y_list) == m and len(s_list) == m:
        y_list.pop(0)
        s_list.pop(0)
        y_list.append(np.array(y))
        s_list.append(np.array(s))
        
    return y_list, s_list
        

In [ ]:
##### INITIAL GUESS #####
M0 = 0.02
M1 = 0.01
write_state(M0, M1)

MAX_ITERS = 50



state = np.array([M0, M1])
gradient = np.array(eval_gradient(M0, M1))

B = np.eye(2, dtype = float)
B_inv = np.eye(2, dtype = float)

s_list = []
y_list = []

m = 10

for iteration in range(MAX_ITERS):
    state_old = np.copy(state)
    gradient_old = np.copy(gradient)
    
    if iteration == 0:
        rho = -np.matmul(B_inv, gradient_old)
        
    else: 

        rho = L_BFGS_update(iteration, state_old, y_list, s_list, m)
        
    alpha = search_alpha(state_old, gradient_old, rho)
    s = alpha*rho
    state = state_old + s
    gradient = eval_gradient(state[0], state[1])
    y = gradient - gradient_old
        
    y_list, s_list = manage_y_s_lists(m, y_list, s_list, y, s)
    
    print(f"iter = {iteration+1}; M0 = {state[0]}; M1 = {state[1]}; Loss = {eval_loss(state[0], state[1])}")
    

iter = 1; M0 = 0.02033114380278951; M1 = 0.0009822981888490488; Loss = 0.4963958949023225
0 -9 0
iter = 2; M0 = 0.020325979150718442; M1 = 0.0011227234332402186; Loss = 0.2880680156888957
0 -8 0
1 -8 0
iter = 3; M0 = 0.020325824419222017; M1 = 0.0011220775201944832; Loss = 0.2880566727551021
0 -7 0
1 -7 0
2 -7 0
iter = 4; M0 = 0.020325672668254635; M1 = 0.0011222171620638892; Loss = 0.2880526316434578
0 -6 0
1 -6 0
2 -6 0
3 -6 0
iter = 5; M0 = 0.020325070760934736; M1 = 0.0011223733248948327; Loss = 0.2880449403181858
0 -5 0
1 -5 0
2 -5 0
3 -5 0
4 -5 0
iter = 6; M0 = 0.02032314357054004; M1 = 0.001122542733858776; Loss = 0.2880258151257073
0 -4 0
1 -4 0
2 -4 0
3 -4 0
4 -4 0
5 -4 0
iter = 7; M0 = 0.020317072133122147; M1 = 0.0011226523928726418; Loss = 0.287972444275405
0 -3 0
1 -3 0
2 -3 0
3 -3 0
4 -3 0
5 -3 0
6 -3 0
iter = 8; M0 = 0.020299760409003442; M1 = 0.0011223529542118914; Loss = 0.28782816048366766
0 -2 0
1 -2 0
2 -2 0
3 -2 0
4 -2 0
5 -2 0
6 -2 0
7 -2 0
iter = 9; M0 = 0.020249